In [1]:
import os
import pandas as pd
# Define the base path and the range of years
base_path = '../team/'
start_year = 1997
end_year = 2025
frames = []
# Loop through each year and list files in the corresponding directory
for year in range(start_year, end_year + 1):
    # Construct the path for the current year
    year_path = os.path.join(base_path, str(year))
    
    # Check if the directory exists
    if os.path.isdir(year_path):
        print(f"\nFiles in directory {year_path}:")
        
        # List all files in the directory
        for filename in os.listdir(year_path):
            file_path = os.path.join(year_path, filename)
            # Check if it is a file before printing
            if 'vs' not in filename and 'avg' not in filename:
                if '.csv' in filename:
                    
                    
                    df=pd.read_csv(file_path)
                    if len(df)>0:
                        print(filename)
                        df['year']=year
                        frames.append(df)

    else:
        print(f"\nDirectory {year_path} does not exist.")

master=pd.concat(frames)
master


Files in directory ../team/1997:
1610612756.csv
1610612765.csv
1610612757.csv
1610612749.csv
1610612738.csv
1610612747.csv
1610612753.csv
1610612742.csv
1610612743.csv
1610612754.csv
1610612766.csv
1610612759.csv
1610612746.csv
1610612763.csv
1610612764.csv
1610612748.csv
1610612737.csv
1610612760.csv
1610612762.csv
1610612752.csv
1610612741.csv
1610612750.csv
1610612751.csv
1610612739.csv
1610612745.csv
1610612744.csv
1610612758.csv
1610612761.csv
1610612755.csv

Files in directory ../team/1998:
1610612756.csv
1610612765.csv
1610612757.csv
1610612749.csv
1610612738.csv
1610612747.csv
1610612753.csv
1610612742.csv
1610612743.csv
1610612754.csv
1610612766.csv
1610612759.csv
1610612746.csv
1610612763.csv
1610612764.csv
1610612748.csv
1610612737.csv
1610612760.csv
1610612762.csv
1610612752.csv
1610612741.csv
1610612750.csv
1610612751.csv
1610612739.csv
1610612745.csv
1610612744.csv
1610612758.csv
1610612761.csv
1610612755.csv

Files in directory ../team/1999:
1610612756.csv
1610612765.cs

,PLAYER_ID,PLAYER_NAME,SHOT_ID,GRID_TYPE,GAME_ID,GAME_EVENT_ID,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,...,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,PLAYERS_ON,year,time,extra
0,208,Sam Cassell,296000123,Shot Chart Detail,29600012,3,1610612756,Phoenix Suns,1,11,...,138,1,0,19961101,LAL,PHX,|109|170|208|445|920,1997,NaN,NaN
1,170,Joe Kleine,2960001214,Shot Chart Detail,29600012,14,1610612756,Phoenix Suns,1,10,...,45,1,0,19961101,LAL,PHX,|109|170|208|445|920,1997,NaN,NaN
2,445,Wesley Person,2960001217,Shot Chart Detail,29600012,17,1610612756,Phoenix Suns,1,9,...,158,1,0,19961101,LAL,PHX,|109|170|208|445|920,1997,NaN,NaN
3,445,Wesley Person,2960001219,Shot Chart Detail,29600012,19,1610612756,Phoenix Suns,1,9,...,212,1,0,19961101,LAL,PHX,|109|170|208|445|920,1997,NaN,NaN
4,445,Wesley Person,2960001229,Shot Chart Detail,29600012,29,1610612756,Phoenix Suns,1,8,...,0,1,1,19961101,LAL,PHX,|109|170|208|445|920,1997,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1630231,KJ Martin,22400179590,Shot Chart Detail,22400179,590,1610612755,Philadelphia 76ers,4,2,...,50,1,1,20241106,LAC,PHI,202704|1630231|1641737|1641741|1642272,2025,27400.0,0.0
596,1641741,Ricky Council IV,22400179596,Shot Chart Detail,22400179,596,1610612755,Philadelphia 76ers,4,1,...,43,1,1,20241106,LAC,PHI,202704|1630231|1641737|1641741|1642272,2025,27690.0,0.0
597,1641741,Ricky Council IV,22400179600,Shot Chart Detail,22400179,600,1610612755,Philadelphia 76ers,4,1,...,31,1,0,20241106,LAC,PHI,202704|1630231|1641737|1641741|1642272,2025,27870.0,0.0
598,1630231,KJ Martin,22400179603,Shot Chart Detail,22400179,603,1610612755,Philadelphia 76ers,4,0,...,36,1,1,20241106,LAC,PHI,202704|1630231|1641737|1641741|1642272,2025,28270.0,0.0


In [2]:
master['SHOT_ATTEMPTED_FLAG'].value_counts()
master['SHOT_MADE_FLAG'].value_counts()

0    3005830
1    2510503
Name: SHOT_MADE_FLAG, dtype: int64

In [3]:
shots = master.groupby(['year','SHOT_ZONE_RANGE','SHOT_DISTANCE']).sum(numeric_only=True)[['SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG']].reset_index()

shots.rename(columns={'SHOT_ATTEMPTED_FLAG':'FGA','SHOT_MADE_FLAG':'FGM'},inplace=True)
shots

,year,SHOT_ZONE_RANGE,SHOT_DISTANCE,FGA,FGM
0,1997,16-24 ft.,16,6681,2734
1,1997,16-24 ft.,17,6614,2725
2,1997,16-24 ft.,18,5948,2437
3,1997,16-24 ft.,19,4944,2143
4,1997,16-24 ft.,20,2955,1466
...,...,...,...,...,...
2754,2025,Less Than 8 ft.,4,786,341
2755,2025,Less Than 8 ft.,5,643,269
2756,2025,Less Than 8 ft.,6,544,210
2757,2025,Less Than 8 ft.,7,463,188


In [4]:
for year in range(1997,2025):
    shot_distance=shots[shots.year==year].reset_index()
    shot_distance.drop(columns='year',inplace=True)
    shot_distance.to_csv(str(year)+'.csv',index=False)

In [5]:
master['SHOT_ZONE_RANGE'].unique()

array(['8-16 ft.', '16-24 ft.', 'Less Than 8 ft.', '24+ ft.',
       'Back Court Shot'], dtype=object)